# 🚀 NSSS Colab Quick Start

**One-Click Setup:** Run the cell below to deploy the NSSS AI Server on Google Colab.

**Requirements:**
1. **Runtime:** GPU (T4 or better) - Go to `Runtime > Change runtime type > T4 GPU`
2. **Ngrok Token:** Get yours at https://dashboard.ngrok.com/get-started/your-authtoken

**What this does:**
- Clones the NSSS repository
- Installs all dependencies (Unsloth, FastAPI, etc.)
- Launches the AI server with Ngrok tunnel
- Provides a public URL for remote access

---

### 📝 Instructions:
1. Replace `YOUR_NGROK_TOKEN_HERE` with your actual token below
2. Press `Shift + Enter` to run
3. Wait ~5 minutes for setup (first time only)
4. Copy the Public URL shown at the end
5. Test with: `python scripts/test_inference_api.py --url <PUBLIC_URL>`

In [ ]:
# ruff: noqa
# ==================== CONFIGURATION ====================
NGROK_AUTHTOKEN = "YOUR_NGROK_TOKEN_HERE"  # 👈 REPLACE THIS
GITHUB_REPO = "https://github.com/Hieureal1305/Neuro-Symbolic_Software_Security.git"

# ==================== AUTO SETUP ====================
import os
import subprocess


def run(cmd):
    """Execute shell command and print output."""
    print(f"⚡ {cmd}")
    result = subprocess.run(cmd, shell=True, capture_output=False, text=True)
    if result.returncode != 0:
        print(f"⚠️  Command failed with code {result.returncode}")
    return result.returncode


# Step 1: Clone Repository
if not os.path.exists("/content/NSSS"):
    print("📥 Cloning NSSS repository...")
    run(f"git clone {GITHUB_REPO} /content/NSSS")
else:
    print("✅ Repository already cloned. Pulling latest changes...")
    run("cd /content/NSSS && git pull")

os.chdir("/content/NSSS")
print(f"📍 Working directory: {os.getcwd()}")

# Step 2: Install Unsloth (Optimized for Colab)
print("\n📦 Installing Unsloth and AI dependencies...")
try:
    import unsloth  # noqa: F401

    print("✅ Unsloth already installed.")
except ImportError:
    run(
        'pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q'
    )
    run(
        'pip install --no-deps "xformers<0.0.27" "trl<0.8.6" peft accelerate bitsandbytes -q'
    )

# Step 3: Install Server Dependencies
print("\n📦 Installing server dependencies...")
run("pip install fastapi uvicorn pyngrok python-multipart nest-asyncio -q")

# Step 4: Install Project Requirements
if os.path.exists("requirements.txt"):
    print("\n📦 Installing project requirements...")
    run("pip install -r requirements.txt -q")

# Step 5: Configure Environment
print("\n🔧 Configuring environment...")
os.environ["NGROK_AUTHTOKEN"] = NGROK_AUTHTOKEN
os.environ["LLM_PROVIDER"] = "local"  # Use local GGUF model
os.environ["API_KEY"] = "nsss-demo-key-2025"  # Default API key

# Step 6: Setup Ngrok
print("\n🔐 Configuring Ngrok tunnel...")
from pyngrok import ngrok

if NGROK_AUTHTOKEN == "YOUR_NGROK_TOKEN_HERE":
    print("⚠️  WARNING: You haven't set your Ngrok token!")
    print("   Get one at: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   The server will still start but the tunnel may expire quickly.")
else:
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    print("✅ Ngrok authenticated.")

# Kill existing tunnels
ngrok.kill()

# Step 7: Start Ngrok Tunnel
PORT = 8000
print(f"\n🔗 Opening tunnel on port {PORT}...")
public_url = ngrok.connect(PORT).public_url

print("\n" + "=" * 60)
print("🎉 NSSS SERVER IS READY!")
print("=" * 60)
print(f"\n📡 Public URL: {public_url}")
print(f"🔍 Health Check: {public_url}/health")
print(f"🧪 Analyze Endpoint: {public_url}/analyze")
print(f"🔑 API Key: nsss-demo-key-2025")
print("\n💡 Test from your laptop:")
print(f"   python scripts/test_inference_api.py --url {public_url}")
print("\n⚠️  Keep this cell running! Server will stop if you interrupt.")
print("=" * 60 + "\n")

# Step 8: Launch Server
print("🔥 Starting FastAPI server...\n")
import nest_asyncio
import uvicorn

nest_asyncio.apply()  # Allow nested event loops in Colab

# Import the FastAPI app
from src.server.colab_server import app

# Run server (blocking - will run until interrupted)
uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="info")

---

## 🎯 Next Steps

Once the server is running:

1. **Test the API** from your local machine:
   ```bash
   git clone https://github.com/Hieureal1305/Neuro-Symbolic_Software_Security.git
   cd Neuro-Symbolic_Software_Security
   pip install requests
   python scripts/test_inference_api.py --url <YOUR_PUBLIC_URL>
   ```

2. **Run Full Analysis** on a vulnerable sample:
   ```bash
   python -m src.runner.cli.main analyze samples/vulnerable_app.py \
       --provider remote \
       --remote-url <YOUR_PUBLIC_URL> \
       --api-key nsss-demo-key-2025
   ```

3. **View Results:**
   - SARIF: `outputs/nsss_report.sarif`
   - Markdown: `outputs/nsss_report.md`
   - Graph: `outputs/nsss_graph.json`

---

## 📚 Documentation

- **Full Demo Walkthrough:** `docs/05b_Demo_Walkthrough.md`
- **API Protocol:** `docs/12_Inference_Protocol.md`
- **Graph Visualization:** `docs/14_Graph_Visualization_Spec.md`
- **Advanced Notebook:** `notebooks/NSSS_Colab_Runner.ipynb` (multi-cell, configurable)

---

## 🐛 Troubleshooting

**Server won't start:**
- Check GPU is enabled: `Runtime > Change runtime type > T4 GPU`
- Verify Ngrok token is correct
- Check logs above for specific errors

**Slow first run:**
- First run downloads ~7GB model (one-time only)
- Subsequent runs use cached model

**API errors:**
- Verify API key matches: `nsss-demo-key-2025`
- Check `/health` endpoint first
- Review server logs in this notebook

---

**Need help?** Open an issue at https://github.com/Hieureal1305/Neuro-Symbolic_Software_Security/issues